In [89]:
import pandas as pd
import numpy as np
# import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# import torch
# import torch.nn as nn

# from deepctr_torch.inputs import SparseFeat, get_feature_names
# from deepctr_torch.models import DeepFM

In [90]:
from rs_datasets import MovieLens
ml = MovieLens()
ml.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992



links


,item_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [91]:
ml.items = pd.concat([ml.items[['item_id', 'title']], ml.items['genres'].str.get_dummies()], axis = 1)
ml.ratings.columns = ['user_id', 'item_id', 'rating', 'rating_timestamp']

In [92]:
data = ml.ratings.join(ml.items.set_index('item_id'), on='item_id').join(ml.links.set_index('item_id'), on='item_id').join(ml.tags.set_index(['item_id', 'user_id']), on=['item_id', 'user_id'])

def process_data(data):
    le1 = LabelEncoder()
    le1.fit(data['tag'])
    data['tag'] = le1.transform(data['tag'])
    
#     le2 = LabelEncoder()
#     le2.fit(data['title'])
#     data['title'] = le1.transform(data['title'])
            
    return data

data = process_data(data)
data.head()

,user_id,item_id,rating,rating_timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,imdb_id,tmdb_id,tag,timestamp
0,1,1,4.0,964982703,Toy Story (1995),0,0,1,1,1,...,0,0,0,0,0,0,114709,862.0,1543,NaN
1,1,3,4.0,964981247,Grumpier Old Men (1995),0,0,0,0,0,...,0,1,0,0,0,0,113228,15602.0,1543,NaN
2,1,6,4.0,964982224,Heat (1995),0,1,0,0,0,...,0,0,0,1,0,0,113277,949.0,1543,NaN
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),0,0,0,0,0,...,1,0,0,1,0,0,114369,807.0,1543,NaN
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",0,0,0,0,0,...,1,0,0,1,0,0,114814,629.0,1543,NaN


In [93]:
data.dtypes

user_id                 int32
item_id                 int32
rating                float64
rating_timestamp        int32
title                  object
(no genres listed)      int64
Action                  int64
Adventure               int64
Animation               int64
Children                int64
Comedy                  int64
Crime                   int64
Documentary             int64
Drama                   int64
Fantasy                 int64
Film-Noir               int64
Horror                  int64
IMAX                    int64
Musical                 int64
Mystery                 int64
Romance                 int64
Sci-Fi                  int64
Thriller                int64
War                     int64
Western                 int64
imdb_id                 int32
tmdb_id               float64
tag                     int64
timestamp             float64
dtype: object

In [94]:
# import pandas as pd
pd.isnull(data).any()

user_id               False
item_id               False
rating                False
rating_timestamp      False
title                 False
(no genres listed)    False
Action                False
Adventure             False
Animation             False
Children              False
Comedy                False
Crime                 False
Documentary           False
Drama                 False
Fantasy               False
Film-Noir             False
Horror                False
IMAX                  False
Musical               False
Mystery               False
Romance               False
Sci-Fi                False
Thriller              False
War                   False
Western               False
imdb_id               False
tmdb_id                True
tag                   False
timestamp              True
dtype: bool